# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
# Load data from cities.csv
cities_df = pd.read_csv("output_data/cities.csv", encoding="utf-8")
cities_df.head()

,City,Cloudiness,Country,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,Shimoda,75,jp,68,34.67,138.95,65.57,19.46
1,Stolin,53,by,72,51.89,26.85,38.14,11.68
2,Albany,75,us,25,42.60,-73.97,57.51,1.88
3,Moranbah,0,au,53,-22.00,148.05,75.54,15.48
4,Maputo,40,mz,73,-25.97,32.59,72.64,10.29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
# Create Humity Heatmap
# Pull in key
gmaps.configure(api_key=g_key)
# Get Locations
locations = cities_df[["Latitude","Longitude"]].astype(float)
humidity = cities_df["Humidity"].astype(float)
maxhumidity = humidity.max()

In [14]:
center = cities_df[["Latitude","Longitude"]].median()
fig = gmaps.figure(center=center, zoom_level=2)
humidity_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                     max_intensity=maxhumidity, point_radius = 3)
fig.add_layer(humidity_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
narrowed_city_df = cities_df.loc[(cities_df["Max Temperature"] > 70) & (cities_df["Max Temperature"] < 80) & (cities_df["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,City,Cloudiness,Country,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,Moranbah,0,au,53,-22.00,148.05,75.54,15.48
1,Rikitea,0,pf,76,-23.12,-134.97,76.17,15.95
2,Atar,0,mr,19,20.52,-13.05,73.00,7.94
3,Guerrero Negro,0,mx,36,27.98,-114.06,77.95,15.48
4,Arraial do Cabo,0,br,80,-22.97,-42.03,75.88,17.16


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
hotels_df = narrowed_city_df.copy()
hotels_df["Hotel Name"] = ""

In [18]:
params = {
    "location": "",
    "keyword":"hotel",
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotels_df.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    params["location"]=f"{latitude},{longitude}"
    response = requests.get(base_url, params=params).json()
    if len(response['results']) > 0:
        hotel_name = response['results'][0]['name']
    else:
        hotel_name = "No Hotel Found"
    hotels_df.loc[index, 'Hotel Name'] = hotel_name

In [19]:
hotels_df

,City,Cloudiness,Country,Humidity,Latitude,Longitude,Max Temperature,Wind Speed,Hotel Name
0,Moranbah,0,au,53,-22.00,148.05,75.54,15.48,Direct Hotels - Monterey Moranbah
1,Rikitea,0,pf,76,-23.12,-134.97,76.17,15.95,Pension Maro'i
2,Atar,0,mr,19,20.52,-13.05,73.00,7.94,Odar kanawal
3,Guerrero Negro,0,mx,36,27.98,-114.06,77.95,15.48,Hotel Los Caracoles
4,Arraial do Cabo,0,br,80,-22.97,-42.03,75.88,17.16,Hotel da Canoa
5,San Quintín,0,mx,51,30.48,-115.95,73.98,6.98,Hotel Jardines Baja
6,Grand Gaube,0,mu,93,-20.01,57.66,75.99,1.01,Lagoon Attitude
7,Keti Bandar,0,pk,84,24.14,67.45,79.45,12.88,No Hotel Found
8,Adrar,0,mr,15,20.50,-10.07,78.67,8.75,No Hotel Found
9,Cockburn Town,0,tc,81,21.46,-71.14,79.61,19.39,Osprey Beach Hotel


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
hotel_locations = hotels_df[["Latitude", "Longitude"]]

In [25]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))